In [1]:
import polars as pl
import json

from generators.loan_generator import LoanGenerator
from generators.inq_generator import InqGenerator
from generators.overview_generator import OverviewGenerator


%load_ext autoreload
%autoreload 2

In [2]:
LOAN_PATH     = "/mnt/nfs/NRazuvaev/[DS-493] CASH.PD.FA.V092023/LOAN.parquet"
INQ_PATH      = '/mnt/nfs/NRazuvaev/[DS-493] CASH.PD.FA.V092023/INQUIRY.parquet'
OVERVIEW_PATH = "/mnt/nfs/NRazuvaev/[DS-493] CASH.PD.FA.V092023/OVERVIEW.parquet"
    
loan_generator     = LoanGenerator(id_column='REQUESTID', depth_filters=3, verbose=True, n_jobs=4)
inquiry_generator  = InqGenerator(id_column='REQUESTID', depth_filters=3, verbose=True, n_jobs=4)
overview_generator = OverviewGenerator(id_column='REQUESTID', depth_filters=3, verbose=True, n_jobs=4)

In [3]:
ids = pl.read_parquet(LOAN_PATH, use_pyarrow=True, columns=['REQUESTID'])['REQUESTID'].unique().sample(1_000)

In [4]:
data_loan     = pl.read_parquet(LOAN_PATH, use_pyarrow=True).filter(pl.col('REQUESTID').is_in(ids))
data_inquiry  = pl.read_parquet(INQ_PATH, use_pyarrow=True).filter(pl.col('REQUESTID').is_in(ids))\
                .with_columns(
                    pl.lit(None).alias('INQ_TYPE'),
                    pl.lit(None).alias('INQ_DATE'),
                )
data_overview = pl.read_parquet(OVERVIEW_PATH, use_pyarrow=True).filter(pl.col('REQUESTID').is_in(ids))

In [5]:
features_loan = loan_generator.generate_mask(data_loan, .01).transform(data_loan)
features_loan.head()

shape: (5, 115_025)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ REQUESTID ┆ last_EST_ ┆ count_EST ┆ sum_EST_N ┆ … ┆ max_PMT_S ┆ min_PMT_S ┆ q25_PMT_S ┆ q75_PMT_ │
│ ---       ┆ NEXT_PMT  ┆ _NEXT_PMT ┆ EXT_PMT   ┆   ┆ TRING_84M ┆ TRING_84M ┆ TRING_84M ┆ STRING_8 │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ _CNT_REFI ┆ _CNT_REFI ┆ _CNT_REFI ┆ 4M_CNT_R │
│           ┆ f32       ┆ u32       ┆ f32       ┆   ┆ N_n…      ┆ N_n…      ┆ N_n…      ┆ EFIN_n…  │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ u32       ┆ u32       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ -24976608 ┆ 18650.0   ┆ 25        ┆ 78650.109 ┆ … ┆ 0         ┆ 0         ┆ 0.0       ┆ 0.0      │
│           ┆           ┆           ┆ 375       ┆   ┆           ┆           ┆           ┆          │
│ -9297769  ┆ 2263.8500 ┆ 4         ┆ 25727.429 ┆ … ┆ 0         ┆ 0         ┆ 0.0       ┆ 0.0      │
│           ┆ 98        ┆           ┆ 688       ┆   ┆           ┆           ┆           ┆          │
│ 503665971 ┆ 1757.3100 ┆ 14        ┆ 43870.007 ┆ … ┆ null      ┆ null      ┆ null      ┆ null     │
│ 32        ┆ 59        ┆           ┆ 812       ┆   ┆           ┆           ┆           ┆          │
│ 106106931 ┆ 1347.8900 ┆ 19        ┆ 140885.48 ┆ … ┆ 0         ┆ 0         ┆ 0.0       ┆ 0.0      │
│ 918       ┆ 15        ┆           ┆ 4375      ┆   ┆           ┆           ┆           ┆          │
│ -33274945 ┆ 8072.0600 ┆ 8         ┆ 34717.777 ┆ … ┆ null      ┆ null      ┆ null      ┆ null     │
│           ┆ 59        ┆           ┆ 344       ┆   ┆           ┆           ┆           ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [6]:
features_inquiry = inquiry_generator.generate_mask(data_inquiry, .01).transform(data_inquiry)
features_inquiry.head()

shape: (5, 4_561)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ REQUESTID ┆ last_INQ_ ┆ count_INQ ┆ sum_INQ_A ┆ … ┆ max_INQ_Q ┆ min_INQ_Q ┆ q25_INQ_Q ┆ q75_INQ_ │
│ ---       ┆ AMT       ┆ _AMT      ┆ MT        ┆   ┆ UANTITY_n ┆ UANTITY_n ┆ UANTITY_n ┆ QUANTITY │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ otMTSB_36 ┆ otMTSB_36 ┆ otMTSB_36 ┆ _notMTSB │
│           ┆ f32       ┆ u32       ┆ f32       ┆   ┆ 5D_…      ┆ 5D_…      ┆ 5D_…      ┆ _365D_…  │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ i32       ┆ i32       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 559248794 ┆ 0.0       ┆ 50        ┆ 6.4637288 ┆ … ┆ 13        ┆ 1         ┆ 1.0       ┆ 2.0      │
│ 79        ┆           ┆           ┆ e7        ┆   ┆           ┆           ┆           ┆          │
│ 116746361 ┆ 150000.0  ┆ 14        ┆ 1.037e7   ┆ … ┆ 1         ┆ 1         ┆ 1.0       ┆ 1.0      │
│ 277       ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ -65051833 ┆ 0.0       ┆ 61        ┆ 3.2153982 ┆ … ┆ 57        ┆ 1         ┆ 1.0       ┆ 4.0      │
│           ┆           ┆           ┆ e7        ┆   ┆           ┆           ┆           ┆          │
│ -46848230 ┆ 0.0       ┆ 114       ┆ 6.8279536 ┆ … ┆ 14        ┆ 1         ┆ 1.0       ┆ 1.0      │
│           ┆           ┆           ┆ e7        ┆   ┆           ┆           ┆           ┆          │
│ 770273343 ┆ 50000.0   ┆ 78        ┆ 3.9769028 ┆ … ┆ 12        ┆ 1         ┆ 1.0       ┆ 1.0      │
│ 88        ┆           ┆           ┆ e7        ┆   ┆           ┆           ┆           ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [7]:
features_overview = overview_generator.generate_mask(data_overview, .01).transform(data_overview)
features_overview.head()

shape: (5, 33)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ REQUESTID ┆ max_LOAN_ ┆ max_TTL_A ┆ max_TTL_C ┆ … ┆ max_IS_CR ┆ max_IS_CR ┆ max_IS_CR ┆ max_IS_C │
│ ---       ┆ MAIN_BORR ┆ CCOUNTS   ┆ ONSUMER   ┆   ┆ EDS_30_59 ┆ EDS_30_59 ┆ EDS_5_29_ ┆ REDS_ACT │
│ str       ┆ OWER      ┆ ---       ┆ ---       ┆   ┆ _TWICE_12 ┆ _ONCE_12M ┆ 3TIMES_12 ┆ IVE_CRED │
│           ┆ ---       ┆ f32       ┆ f32       ┆   ┆ M         ┆ ---       ┆ M         ┆ S_6M     │
│           ┆ f32       ┆           ┆           ┆   ┆ ---       ┆ f32       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ f32       ┆           ┆ f32       ┆ f32      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ -94384719 ┆ 14.0      ┆ 14.0      ┆ 9.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 1.0       ┆ 1.0      │
│ -92564995 ┆ 10.0      ┆ 10.0      ┆ 7.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 1.0       ┆ 1.0      │
│ 954142546 ┆ 30.0      ┆ 30.0      ┆ 19.0      ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 1.0      │
│ 96        ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ -58312285 ┆ 10.0      ┆ 10.0      ┆ 6.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 1.0       ┆ 1.0      │
│ -56607160 ┆ 11.0      ┆ 11.0      ┆ 10.0      ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 1.0      │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [8]:
features = pl.concat([features_loan, features_inquiry, features_overview], how='align')
features

shape: (1_000, 119_617)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ REQUESTID ┆ last_EST_ ┆ count_EST ┆ sum_EST_N ┆ … ┆ max_IS_CR ┆ max_IS_CR ┆ max_IS_CR ┆ max_IS_C │
│ ---       ┆ NEXT_PMT  ┆ _NEXT_PMT ┆ EXT_PMT   ┆   ┆ EDS_30_59 ┆ EDS_30_59 ┆ EDS_5_29_ ┆ REDS_ACT │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ _TWICE_12 ┆ _ONCE_12M ┆ 3TIMES_12 ┆ IVE_CRED │
│           ┆ f32       ┆ u32       ┆ f32       ┆   ┆ M         ┆ ---       ┆ M         ┆ S_6M     │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ f32       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ f32       ┆           ┆ f32       ┆ f32      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ -10022833 ┆ 0.1       ┆ 20        ┆ 318010.28 ┆ … ┆ null      ┆ null      ┆ null      ┆ null     │
│           ┆           ┆           ┆ 125       ┆   ┆           ┆           ┆           ┆          │
│ -10026556 ┆ 1695.7600 ┆ 20        ┆ 46921.667 ┆ … ┆ null      ┆ null      ┆ null      ┆ null     │
│           ┆ 1         ┆           ┆ 969       ┆   ┆           ┆           ┆           ┆          │
│ -10064749 ┆ 2000.0    ┆ 14        ┆ 144505.59 ┆ … ┆ null      ┆ null      ┆ null      ┆ null     │
│           ┆           ┆           ┆ 375       ┆   ┆           ┆           ┆           ┆          │
│ -10155648 ┆ 9500.0    ┆ 5         ┆ 23723.990 ┆ … ┆ null      ┆ null      ┆ null      ┆ null     │
│           ┆           ┆           ┆ 234       ┆   ┆           ┆           ┆           ┆          │
│ -10246405 ┆ 6176.5297 ┆ 8         ┆ 29582.529 ┆ … ┆ null      ┆ null      ┆ null      ┆ null     │
│           ┆ 85        ┆           ┆ 297       ┆   ┆           ┆           ┆           ┆          │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …        │
│ 984382879 ┆ 2028.8299 ┆ 4         ┆ 9033.25   ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 1.0      │
│ 43        ┆ 56        ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 985361505 ┆ 7932.6899 ┆ 12        ┆ 37841.531 ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 1.0      │
│ 14        ┆ 41        ┆           ┆ 25        ┆   ┆           ┆           ┆           ┆          │
│ 987288571 ┆ 845.71002 ┆ 30        ┆ 134095.48 ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 1.0      │
│ 19        ┆ 2         ┆           ┆ 4375      ┆   ┆           ┆           ┆           ┆          │
│ 990366210 ┆ 7833.0400 ┆ 11        ┆ 141849.56 ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 1.0      │
│ 35        ┆ 39        ┆           ┆ 25        ┆   ┆           ┆           ┆           ┆          │
│ 991110715 ┆ 19271.160 ┆ 12        ┆ 99927.437 ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 1.0      │
│ 72        ┆ 156       ┆           ┆ 5         ┆   ┆           ┆           ┆           ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘